In [1]:
from xhtml2pdf import pisa
from xhtml2pdf.config.httpconfig import httpConfig

In [2]:
# State

In [3]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
start_all = time.time()

In [6]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
tester_objects = [povm_x, povm_y, povm_z]

In [7]:
true_object = get_z0_1q(c_sys)
num_data = [100, 1000]
n_rep = 400

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

qtomography_list = [
    StandardQst(tester_objects, on_para_eq_constraint=True),
    StandardQst(tester_objects, on_para_eq_constraint=False),
    StandardQst(tester_objects, on_para_eq_constraint=True),
    StandardQst(tester_objects, on_para_eq_constraint=False),
]
para_list = [True, False, True, False]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
]

estimation_results_list = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")

  0%|          | 0/400 [00:00<?, ?it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LinearEstimator


  0%|          | 0/400 [00:00<?, ?it/s]


SystemError: <class 'enumerate'> returned a result with an error set

In [ ]:
# MSE
mse_div = report.generate_mse_div(
    estimation_results_list=estimation_results_list,
    case_name_list=case_name_list,
    true_object=true_object,
    num_data=num_data,
    n_rep=n_rep,
    qtomographies=[StandardQst(tester_objects, on_para_eq_constraint=True)]
)

In [ ]:
physicality_violation_test_true_case_divs = ""
physicality_violation_test_false_eigenvalues_divs = ""
physicality_violation_test_false_sum_eigenvalues_divs = ""

for case_id, case_name in enumerate(case_name_list):
    estimation_results = estimation_results_list[case_id]
    if para_list[case_id]:
        # on_para_eq_constraint = True
        div = report.generate_trace_div(estimation_results, case_id=case_id, num_data=num_data)
        physicality_violation_test_true_case_divs += f"""
        <h4>Case {case_id}: {case_name}<h4>
            {div}
        """
    else:
        # on_para_eq_constraint = False
        div = report.generate_eigenvalues_div(estimation_results, case_id=case_id, num_data=num_data, true_object=true_object)
        physicality_violation_test_false_eigenvalues_divs += f"""
        <h4>Case {case_id}: {case_name}<h4>
        {div}
        """
        
        div = report.generate_sum_eigenvalues_div(estimation_results, case_id=case_id, num_data=num_data, true_object=true_object)
        physicality_violation_test_false_sum_eigenvalues_divs += f"""
        <h4>Case {case_id}: {case_name}<h4>
        {div}
        """

physicality_violation_test_true_all_div = f"""
            <h2>on_para_eq_constraint=True</h2>
            {physicality_violation_test_true_case_divs}
        """
physicality_violation_test_false_all_div = f"""
            <h2>on_para_eq_constraint=False</h2>
            <h3>Eigenvalue</h3>
            {physicality_violation_test_false_eigenvalues_divs}
            <h3>Sum of unphysical eigenvalues </h3>
            {physicality_violation_test_false_sum_eigenvalues_divs}
        """

physicality_violation_test_div = f"""
{physicality_violation_test_div_true}
{physicality_violation_test_div_false}
"""

In [ ]:
# True Object and Tester Objects
true_object_table = report._convert_object_to_datafrane(true_object).to_html(
    classes="true_object_table", escape=False, header=False
)
tester_table = report._convert_objects_to_multiindex_dataframe(tester_objects).to_html(
    classes="tester_objects_table", escape=False, header=False
)

In [ ]:
# Experiment Condition
type_tomography_values = list(set([qt.__class__.__name__ for qt in qtomography_list]))

info = {
    "Type of tomography": type_tomography_values,
    "Nrep": [iteration],
    "N": [num_data],
}
condition_df = pd.DataFrame(info).T
condition_table = condition_df.to_html(
    classes="condition_table", escape=False, header=False
)

In [ ]:
# Cases
case_dict = dict(
    Name=case_name_list,
    Parameterization=p_list,
    Tomography=[t.__class__.__name__ for t in qtomography_list],
    Estimator=[e.__class__.__name__ for e in estimator_list],
)

df = pd.DataFrame(case_dict)

styles = [
    dict(selector=".col0", props=[("width", "400px")]),
    dict(selector=".col1", props=[("width", "200px")]),
    dict(selector=".col2", props=[("width", "200px")])
]
case_table = df.style.set_table_styles(styles).render()

In [ ]:
report_html = f"""<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    
    <style type="text/css">
        <!--
            {report._css}
            {report._inline_block_css}
            {report._table_css}
            {report._table_contents_css}
         -->
    </style>
<title>Quara Report</title>
</head>
<body>
<div id="table_of_contents">
    <h1>Table of contents</h1>
    <pdf:toc />
</div>
<h1>Experimental condition</h1>
    <div>
        {condition_table}
    </div>
<h2>True object</h2>
    <div>
        {true_object_table}
    </div>
<h2>Tester objects</h2>
    <div>
        {tester_table}
    </div>
<h2>Cases</h2>
    <div>
        {case_table}
    </div>
<h1>Consistency test</h1>
    <div>(TODO)</div>
<h1>MSE</h1>
    <div>
    {mse_div}
    </div>
<h1>Physicality violation test</h1>
    <div>
        {physicality_violation_test_true_all_div}
        {physicality_violation_test_false_all_div }
    </div>
</body>
</html>"""

In [ ]:
with open("test.html", "w") as f:
    f.write(report_html)
    
report._convert_html2pdf(report_html, "sample_1.pdf")

In [ ]:
!open test.html

In [ ]:
!open sample_1.pdf

In [ ]:
report.export_report("sample_100.pdf",
             estimation_results_list,
    case_name_list,
    qtomography_list,
    para_list,
    estimator_list,
    true_object,
    tester_objects,
    num_data,
    n_rep,
)

In [ ]:
all_elapsed_time = time.time() - start_all
print("all_elapsed_time:{0}".format(all_elapsed_time / 60) + "[min]")